# Common DEGs and core-accessory genes

This notebook looks to see if the common DEGs, identified using [SOPHIE](https://github.com/greenelab/generic-expression-patterns/blob/master/pseudomonas_analysis/2_identify_generic_genes_pathways.ipynb) are mostly core or accessory genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import pandas as pd
from core_acc_modules import utils, paths

### Get common DEGs

In [2]:
# Read in SOPHIE identified common DEGs (PAO1 gene ids)
common_DEGs_filename = paths.COMMON_DEGS

common_DEGs = pd.read_csv(common_DEGs_filename, sep="\t", index_col=0, header=0)

In [3]:
print(common_DEGs.shape)
common_DEGs.head()

(834, 3)


Percentile (simulated)  prop DEGs gene id
ProbeID                                                  
PA2618_at                    97.566691  90.007215  PA2618
PA3479_rhlA_at               85.940880  98.890693  PA3479
PA0512_at                    96.268926  74.918831  PA0512
PA3187_at                    88.626532  87.626263  PA3187
PA5191_at                    86.571738  74.918831  PA5191

In [4]:
common_DEGs = common_DEGs.set_index("gene id")

### Get core/accessory annotations

In [5]:
# Read in expression data
pao1_expression_filename = paths.PAO1_COMPENDIUM
pa14_expression_filename = paths.PA14_COMPENDIUM

pao1_expression = pd.read_csv(pao1_expression_filename, sep="\t", index_col=0, header=0)
pa14_expression = pd.read_csv(pa14_expression_filename, sep="\t", index_col=0, header=0)

Note: Core and accessory annotations are from [BACTOME](https://academic.oup.com/nar/article/47/D1/D716/5112984). Not all core genes are measured by our expression dataset ("my dataset") we're using, so there may be a difference in "Number of PAO1 core genes" (core genes from BACTOME) and "Number of PAO1 core genes in my dataset" (core genes that are found in my expression dataset.

In [6]:
pao1_annot_filename = paths.GENE_PAO1_ANNOT
pa14_annot_filename = paths.GENE_PA14_ANNOT

core_acc_dict = utils.get_my_core_acc_genes(
    pao1_annot_filename, pa14_annot_filename, pao1_expression, pa14_expression
)

Number of PAO1 core genes: 5366
Number of PA14 core genes: 5363
Number of PAO1 core genes in my dataset: 5361
Number of PA14 core genes in my dataset: 5361
Number of PAO1-specific genes: 202
Number of PA14-specific genes: 530


In [7]:
pao1_core = core_acc_dict["core_pao1"]
pa14_core = core_acc_dict["core_pa14"]
pao1_acc = core_acc_dict["acc_pao1"]
pa14_acc = core_acc_dict["acc_pa14"]

### Are common genes mostly core or accessory?

In [8]:
# Get shared gene ids
shared_acc = common_DEGs.index.intersection(pao1_acc)
shared_core = common_DEGs.index.intersection(pao1_core)

In [9]:
common_DEGs.loc[shared_acc, "gene group"] = "accessory"
common_DEGs.loc[shared_core, "gene group"] = "core"

In [10]:
common_DEGs["gene group"].value_counts()

core         806
accessory     27
Name: gene group, dtype: int64

In [11]:
# Add gene name
pao1_gene_annot = pd.read_csv(pao1_annot_filename, index_col=0, header=0)
pao1_gene_annot = pao1_gene_annot["Name"].to_frame("gene name")

In [12]:
common_DEGs_label = common_DEGs.merge(
    pao1_gene_annot, left_index=True, right_index=True
)

In [13]:
common_DEGs_label

Percentile (simulated)  prop DEGs gene group gene name
PA2618               97.566691  90.007215       core       NaN
PA3479               85.940880  98.890693       core      rhlA
PA0512               96.268926  74.918831       core      nirH
PA3187               88.626532  87.626263       core       NaN
PA5191               86.571738  74.918831       core       NaN
PA0838               94.051911  81.935426       core       NaN
PA1988               98.008291  81.935426       core      pqqD
PA5046               85.634463  87.626263       core       NaN
PA2840               90.374910  92.478355       core       NaN
PA3418               98.954578  99.630231       core       ldh
PA1003               93.249820  70.571789       core      mvfR
PA0162               83.047945  74.918831       core      opdC
PA2146               99.837779  87.626263       core       NaN
PA4630               91.149964  60.750361       core       NaN
PA3348               88.887888  78.715729       core       NaN
PA5409               86.121125  70.571789       core       NaN
PA4432               80.109950  98.385642       core      rpsI
PA0629               95.872386  90.007215       core       NaN
PA5359               97.224225  94.606782       core       NaN
PA1930               98.756309  98.385642       core       NaN
PA2375               99.279019  90.007215       core       NaN
PA4139               83.525595  92.478355       core       NaN
PA2544               86.995314  60.750361       core       NaN
PA4563               85.553353  97.646104       core      rpsT
PA1150               93.204758  74.918831  accessory      pys2
PA3570               83.273252  99.323593       core      mmsA
PA0509               98.044340  87.626263       core      nirN
PA4940               88.860851  94.606782       core       NaN
PA3986               99.351118  97.646104       core       NaN
PA1118               91.411319  78.715729       core       NaN
...                        ...        ...        ...       ...
PA4356               82.335977  65.773810       core      xenB
PA3477               85.553353  97.646104       core      rhlR
PA4333               82.948810  94.606782       core       NaN
PA4143               85.490267  65.773810       core       NaN
PA2633               93.754506  74.918831       core       NaN
PA4017               80.740807  60.750361       core       NaN
PA5429               83.327325  98.890693       core      aspA
PA4228               89.140231  98.385642       core      pchD
PA3496               95.529921  78.715729       core       NaN
PA1095               99.693583  90.007215       core       NaN
PA3330               86.995314  92.478355       core       NaN
PA0175               87.418890  87.626263       core       NaN
PA0622               98.359769  97.646104       core       NaN
PA4131               98.972603  99.630231       core       NaN
PA5554               80.587599  92.478355       core      atpD
PA1656               94.718818  65.773810       core     hsiA2
PA4219               89.708003  84.920635       core      ampO
PA3353               93.249820  92.478355       core       NaN
PA0646               96.196828  87.626263       core       NaN
PA3431               81.894376  78.715729       core       NaN
PA4888               85.075703  74.918831       core      desB
PA4670               89.365537  92.478355       core       prs
PA3559               93.312906  65.773810       core       NaN
PA5312               86.265321  87.626263       core      pauC
PA0546               85.904831  98.385642       core      metK
PA1088               87.653208  74.918831       core       NaN
PA1999               99.333093  98.890693       core      dchA
PA4298               93.574261  65.773810       core       NaN
PA1473               92.132300  70.571789       core       NaN
PA1663               97.007931  60.750361       core      sfa2

[834 rows x 4 columns]

In [14]:
common_DEGs_label.to_csv("common_DEGs_gene_group_labeled.tsv", sep="\t")

**Takeaway:**

Looks like most common DEGs are core, as expected. It is thought that these core genes encode essential functions shared by all strains and so it would make sense that these core genes are also those commonly DEGs.